In [ ]:
!pip install -r requirements.txt

: 

In [18]:
import asyncio
import logging

from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool, ConnectionType
from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

logging.basicConfig(level=logging.WARNING)


In [ ]:
import asyncio
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool, ConnectionType
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

async def search_azure_docs(question: str):
    # 1. Set up Azure connection
    settings = AzureAIAgentSettings.create()
    client = await AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=settings.project_connection_string.get_secret_value()
    ).__aenter__()

    # 2. Find our search connection
    connections = await client.connections.list()
    search_conn = next(
        c.id for c in connections 
        if c.connection_type == ConnectionType.AZURE_AI_SEARCH
    )

    # 3. Create a search tool and agent
    search_tool = AzureAISearchTool(
        index_connection_id=search_conn,
        index_name="azure-search-docs"
    )

    agent = AzureAIAgent(
        client=client,
        definition=await client.agents.create_agent(
            model=settings.model_deployment_name,
            instructions="Help answer questions using Azure AI Search docs",
            tools=search_tool.definitions,
            tool_resources=search_tool.resources,
            headers={"x-ms-enable-preview": "true"}
        )
    )

    # 4. Create chat thread and ask question
    thread = await client.agents.create_thread()
    await agent.add_chat_message(
        thread_id=thread.id,
        message=ChatMessageContent(
            role=AuthorRole.USER,
            content=question
        )
    )

    # 5. Get answers!
    async for response in agent.invoke(thread_id=thread.id):
        if response.role != AuthorRole.TOOL:
            print(f"🤖 {response.content}")

# For Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

# Run it!
print("🔍 Searching Azure docs...")
asyncio.run(search_azure_docs("What is BQ?"))

🔍 Searching Azure docs...
🤖 BQ can refer to several things depending on the context. In the realm of technology, it often stands for "BigQuery," which is a data warehouse solution provided by Google Cloud for storing and analyzing large datasets. It enables users to run SQL-like queries on massive amounts of data.

If you meant BQ in a different context, please provide more details, and I'll be happy to assist further!
